In [1]:
import logging

class CustomFormatter(logging.Formatter):
    BLACK = "\033[0;30m"
    RED = "\033[0;31m"
    GREEN = "\033[0;32m"
    BROWN = "\033[0;33m"
    BLUE = "\033[0;34m"
    PURPLE = "\033[0;35m"
    CYAN = "\033[0;36m"
    LIGHT_GRAY = "\033[0;37m"
    DARK_GRAY = "\033[1;30m"
    LIGHT_RED = "\033[1;31m"
    LIGHT_GREEN = "\033[1;32m"
    YELLOW = "\033[1;33m"
    LIGHT_BLUE = "\033[1;34m"
    LIGHT_PURPLE = "\033[1;35m"
    LIGHT_CYAN = "\033[1;36m"
    LIGHT_WHITE = "\033[1;37m"
    BOLD = "\033[1m"
    FAINT = "\033[2m"
    ITALIC = "\033[3m"
    UNDERLINE = "\033[4m"
    BLINK = "\033[5m"
    NEGATIVE = "\033[7m"
    CROSSED = "\033[9m"
    END = "\033[0m"
    reset = "\x1b[0m"
    format = "%(message)s"

    FORMATS = {
        logging.DEBUG: CYAN + "%(levelname)s" + reset + " - " + format,
        logging.INFO: GREEN + "%(levelname)s" + reset + " - " + format,
        logging.WARNING: YELLOW + "%(levelname)s" + reset + " - " + format,
        logging.ERROR: RED + "%(levelname)s" + reset + " - " + format,
    }

    def format(self, record):
        log_fmt = self.FORMATS.get(record.levelno)
        formatter = logging.Formatter(log_fmt)
        return formatter.format(record)

# Logging Configuration Function
def configure_logging():
    # Get the root logger
    logger = logging.getLogger()
    
    # Remove all existing handlers
    if logger.hasHandlers():
        logger.handlers.clear()
    
    # Initialize the handler with the custom formatter
    handler = logging.StreamHandler()
    handler.setFormatter(CustomFormatter())

    # Set the handler for the logger
    logger.addHandler(handler)
    logger.setLevel(logging.DEBUG)

    return logger

In [2]:
import inspect
import logging
from functools import wraps
from typing import Any, Dict, List, Union

logger = configure_logging()
class InvalidSelectionError(Exception):
    pass


class HP:
    def __init__(self, selections: Dict[str, Any], overrides: Dict[str, Any]):
        self.selections = selections
        self.overrides = overrides
        self.config_dict = {}
        logging.info(f"Initialized HP with selections: {self.selections} and overrides: {self.overrides}")

    def select(self, name: str, options: Union[List, Dict], default: Any = None):
        logging.debug(f"Selecting for {name}")
        logging.debug(f"Options: {options}")
        logging.debug(f"Default: {default}")

        if name in self.selections:
            selected_value = self.selections[name]
            logging.info(f"Found selection for {name}: {selected_value}")
            if isinstance(options, dict):
                if selected_value in options:
                    result = options[selected_value]
                else:
                    raise InvalidSelectionError(
                        f"Invalid selection '{selected_value}' for '{name}'. Not in options: {list(options.keys())}"
                    )
            elif isinstance(options, list):
                if selected_value in options:
                    result = selected_value
                else:
                    raise InvalidSelectionError(
                        f"Invalid selection '{selected_value}' for '{name}'. Not in options: {options}"
                    )
            else:
                raise InvalidSelectionError(f"Invalid selection '{selected_value}' for '{name}'.")
        else:
            if isinstance(options, dict):
                result = next(iter(options.values()))
            elif isinstance(options, list):
                result = options[0]
            else:
                result = default
            logging.info(f"Using default value for {name}: {result}")

        # Apply override if it exists
        if name in self.overrides:
            override_value = self.overrides[name]
            if isinstance(options, (list, dict)) and override_value in options:
                result = options[override_value] if isinstance(options, dict) else override_value
            else:
                result = override_value
            logging.info(f"Applied override for {name}: {result}")

        self.config_dict[name] = result
        return result


def extract_function_body(func):
    source = inspect.getsource(func)
    lines = source.split("\n")
    body_start = next(i for i, line in enumerate(lines) if line.strip().endswith(":"))
    body_lines = lines[body_start + 1 :]
    min_indent = min(len(line) - len(line.lstrip()) for line in body_lines if line.strip())
    return "\n".join(line[min_indent:] for line in body_lines)


def hypster(func):
    @wraps(func)
    def wrapper(final_vars: List[str], selections: Dict[str, Any], overrides: Dict[str, Any]):
        logging.info(f"Wrapper called with final_vars: {final_vars}, selections: {selections}, overrides: {overrides}")
        try:
            hp = HP(selections, overrides)

            # Extract and execute the function body
            function_body = extract_function_body(func)
            exec_globals = func.__globals__.copy()
            exec_globals["hp"] = hp
            local_vars = {}
            exec(function_body, exec_globals, local_vars)

            # Filter out built-in variables and function-specific items
            filtered_vars = {k: v for k, v in local_vars.items() if not k.startswith("__") and k != "hp"}

            # If final_vars is empty, return all variables
            if not final_vars:
                result = filtered_vars
            else:
                result = {k: v for k, v in filtered_vars.items() if k in final_vars}

            logging.debug(f"Result after filtering: {result}")

            return result
        except InvalidSelectionError as e:
            logging.error(str(e))
            return None

    return wrapper


In [3]:
import logging
from dataclasses import dataclass

@dataclass
class Mock:
    output_format: str
    llm_model: str
    prompt_images: bool

@hypster
def config(hp):
    output_format = hp.select("output_format", ["json", "markdown"], default="markdown")
    llm_model = hp.select("llm_model", {"gpt-4o-new": "gpt-4o-2024-08-06", 
                                        "gpt-4o-mini": "gpt-4o-mini"}, 
                                        default="gpt-4o-mini")
    prompt_images = hp.select("prompt_images", [True, False], default=True)
    mock = Mock(output_format=output_format, llm_model=llm_model, prompt_images=prompt_images)
    mock2 = Mock(output_format=output_format, llm_model="hello", prompt_images=prompt_images)

# Usage example
logging.getLogger().setLevel(logging.WARNING)

# Test with empty final_vars
final_vars = []
selections = {}
overrides = {"prompt_images": True, "llm_model": "gp3t-4o-new"}

res = config(final_vars, selections, overrides)
print("Final result (all variables):", res)
# Test with specific final_vars
final_vars = ["mock"]
res = config(final_vars, selections, overrides)
print("Final result (specific variables):", res)

Final result (all variables): {'output_format': 'json', 'llm_model': 'gp3t-4o-new', 'prompt_images': True, 'mock': Mock(output_format='json', llm_model='gp3t-4o-new', prompt_images=True), 'mock2': Mock(output_format='json', llm_model='hello', prompt_images=True)}
Final result (specific variables): {'mock': Mock(output_format='json', llm_model='gp3t-4o-new', prompt_images=True)}
